# Getting all coins from Messari API

In [1]:
# libraries
import json
import requests
import os

## Getting all the tokens from the all time high endpoint

In [2]:
# URL and headers for the Messari API call
url = 'https://api.messari.io/marketdata/v1/assets/ath'

# API key from environment variable
API_KEY = os.getenv('MESSARI_ANDRE_KEY')

headers = {
    'accept': 'application/json',
    'x-messari-api-key': API_KEY,
}

In [3]:
response = requests.get(url, headers=headers)

In [6]:
if response.status_code == 200:
    result = response.json()

    data = result['data']

    with open('../data/raw/all-tokens.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)

    print(f"Information saved in '../data/raw/all-tokens.json'. Number of tokens: {len(data)}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Information saved in '../data/raw/all-tokens.json'. Number of tokens: 14728
